## 날씨 및 인구데이터 처리
(2019 기준)
- 기상청 날씨 데이터
- KOSIS 인구 데이터

(2020.08.21 업데이트)
- KOSIS 인구 데이터(월별 기준, 2019.01 ~ 2020.06)
- 날씨 데이터 2020.01 ~ 2020.06 추가


In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
print("pandas : ", pd.__version__)
print("numpy : ", np.__version__)

pandas :  1.1.1
numpy :  1.19.1


In [2]:
weather = pd.read_csv("data/weather.csv", encoding='cp949', engine='python')
weather.head()

,지점,지점명,일시,기온(°C),강수량(mm),습도(%)
0,90,속초,2019-01-01 01:00,-1.9,NaN,18.0
1,90,속초,2019-01-01 02:00,-2.2,NaN,17.0
2,90,속초,2019-01-01 03:00,-2.5,NaN,18.0
3,90,속초,2019-01-01 04:00,-3.9,NaN,20.0
4,90,속초,2019-01-01 05:00,-4.9,NaN,22.0


In [3]:
weather.isna().sum()

지점              0
지점명             0
일시              0
기온(°C)        131
강수량(mm)    757996
습도(%)         953
dtype: int64

강수량의 경우 결측이 대부분이고 습도로 대신할 수 있을 것으로 판단되기 때문에 제외해준다

In [4]:
weather = weather.drop(['강수량(mm)'], axis=1)

In [5]:
weather.tail()

,지점,지점명,일시,기온(°C),습도(%)
830770,295,남해,2019-12-30 20:00,9.1,71.0
830771,295,남해,2019-12-30 21:00,7.5,64.0
830772,295,남해,2019-12-30 22:00,5.9,50.0
830773,295,남해,2019-12-30 23:00,4.6,54.0
830774,295,남해,2019-12-31 00:00,3.6,55.0


2020년 데이터와 결합해준다.

In [7]:
weather1 = pd.read_csv("data/weather121.csv", encoding = 'cp949', engine = 'python')
display(weather1.head())
display(weather1.tail())

,지점,지점명,일시,기온(°C),습도(%)
0,90,속초,2019-12-31 01:00,-3.3,32
1,90,속초,2019-12-31 02:00,-4.8,33
2,90,속초,2019-12-31 03:00,-4.5,28
3,90,속초,2019-12-31 04:00,-5.2,29
4,90,속초,2019-12-31 05:00,-5.6,29


,지점,지점명,일시,기온(°C),습도(%)
2275,295,남해,2019-12-31 20:00,-1.9,45
2276,295,남해,2019-12-31 21:00,-2.8,51
2277,295,남해,2019-12-31 22:00,-3.2,53
2278,295,남해,2019-12-31 23:00,-4.4,59
2279,295,남해,2020-01-01 00:00,-4.7,61


In [8]:
weather2 = pd.read_csv("data/weather2020.csv", encoding='cp949', engine='python')
weather2.head()

,지점,지점명,일시,기온(°C),습도(%)
0,90,속초,2020-01-01 01:00,-2.5,32.0
1,90,속초,2020-01-01 02:00,-2.5,38.0
2,90,속초,2020-01-01 03:00,-2.0,39.0
3,90,속초,2020-01-01 04:00,-1.7,40.0
4,90,속초,2020-01-01 05:00,-1.1,37.0


In [9]:
weather = pd.concat([weather, weather1]).reset_index(drop = True)
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 833055 entries, 0 to 833054
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   지점      833055 non-null  int64  
 1   지점명     833055 non-null  object 
 2   일시      833055 non-null  object 
 3   기온(°C)  832924 non-null  float64
 4   습도(%)   832102 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 31.8+ MB


In [10]:
weather = pd.concat([weather, weather2]).reset_index(drop = True)
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1245486 entries, 0 to 1245485
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   지점      1245486 non-null  int64  
 1   지점명     1245486 non-null  object 
 2   일시      1245486 non-null  object 
 3   기온(°C)  1245338 non-null  float64
 4   습도(%)   1244521 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 47.5+ MB


In [11]:
weather.isna().sum()

지점          0
지점명         0
일시          0
기온(°C)    148
습도(%)     965
dtype: int64

In [12]:
weather[pd.isnull(weather[['습도(%)']]).any(axis=1)]

,지점,지점명,일시,기온(°C),습도(%)
261303,152,울산,2019-12-02 11:00,7.1,NaN
263606,155,창원,2019-03-09 10:00,10.2,NaN
349441,175,진도(첨찰산),2019-01-12 02:00,-0.1,NaN
349607,175,진도(첨찰산),2019-01-19 00:00,3.8,NaN
349608,175,진도(첨찰산),2019-01-19 01:00,3.5,NaN
...,...,...,...,...,...
924240,131,청주,2020-06-29 02:00,22.2,NaN
1147523,259,강진군,2020-03-18 09:00,4.5,NaN
1147524,259,강진군,2020-03-18 10:00,6.5,NaN
1147525,259,강진군,2020-03-18 11:00,13.2,NaN


시단위였음..

In [13]:
# 아무튼 계속해보자
na = weather[pd.isnull(weather[['기온(°C)', '습도(%)']]).any(axis=1)].index

연속된 시간 혹은 비슷한 시간대에 결측이 일어났으므로 선형으로 비례하는 방식으로 결측값을 보간한다(interpolation)

> ### Pandas - interpolate()
> DataFrame 값에 선형으로 비례하는 방식으로 결측값 보간(method='values')
>> method='time'으로 설정할 경우 시계열 날짜 index를 기준으로 결측값 보간이 가능하다

In [14]:
w1 = weather.interpolate(method='values')

In [15]:
w1.iloc[list(na)]

,지점,지점명,일시,기온(°C),습도(%)
58662,101,춘천,2019-09-19 00:00,18.30,87.0
86212,105,강릉,2019-11-14 08:00,2.92,18.0
86213,105,강릉,2019-11-14 09:00,3.74,16.0
86214,105,강릉,2019-11-14 10:00,4.56,15.0
86215,105,강릉,2019-11-14 11:00,5.38,17.0
...,...,...,...,...,...
1147526,259,강진군,2020-03-18 12:00,15.80,36.8
1169218,264,함양군,2020-03-19 09:00,10.15,44.0
1195261,276,청송군,2020-03-19 10:00,12.90,43.0
1215938,283,경주시,2020-02-05 14:00,0.90,16.0


In [16]:
## 컬럼 명 변경 및 지점 코드 drop
w1 = w1.drop(['지점'], axis=1)
w1.columns = ['loc', 'time', 'temp', 'hum']
w1.head()

,loc,time,temp,hum
0,속초,2019-01-01 01:00,-1.9,18.0
1,속초,2019-01-01 02:00,-2.2,17.0
2,속초,2019-01-01 03:00,-2.5,18.0
3,속초,2019-01-01 04:00,-3.9,20.0
4,속초,2019-01-01 05:00,-4.9,22.0


In [17]:
# 시도별 관측소 처리를 위한 사전
location = {'서울특별시':['관악산', '서울'],
            '부산광역시':['부산'],
            '대구광역시':['대구', '대구(기)'],
            '인천광역시':['강화', '백령도', '인천'],
            '광주광역시':['광주'],
            '대전광역시':['대전'],
            '울산광역시':['울산'],
            '경기도':['동두천', '수원', '양평', '이천', '파주'],
            '강원도':['강릉', '대관령', '동해', '북강릉', '북춘천', '삼척',
                   '속초', '영월', '원주', '인제', '정선군', '철원', '춘천',
                   '태백', '홍천'],
            '충청북도':['보은', '제천', '청주', '추풍령', '충주'],
            '충청남도':['금산', '보령', '부여', '서산', '천안', '홍성'],
            '전라북도': ['고창', '고창군', '군산', '남원', '부안', '순창군', '임실',
                     '장수', '전주', '정읍'],
            '전라남도':['강진군', '고흥', '광양시', '목포', '무안', '보성군',
                    '순천', '여수', '영광군', '완도', '장흥', '주암',
                    '진도(첨찰산)', '진도군', '해남', '흑산도'],
            '경상북도':['경주시', '구미', '문경', '봉화', '상주', '안동', '영덕',
                   '영주', '영천', '울릉도', '울진', '의성', '청송군', '포항'],
            '경상남도':['거제', '거창', '김해시', '남해', '밀양', '북창원', '산청',
                    '양산시', '의령군', '진주', '창원', '통영', '함양군', '합천'],
            '제주도':['고산', '서귀포', '성산', '성산포', '제주'],
            '세종특별자치시':['세종']
           }

In [18]:
def region(x):
    for key, value in location.items():
        if x in value:
            return key

In [19]:
w1['region'] = tqdm(w1['loc'].apply(region))

In [20]:
w1.isna().sum()

loc       0
time      0
temp      0
hum       0
region    0
dtype: int64

잘 처리되었습니다

In [21]:
display(w1.head())
print(w1['region'].unique())

,loc,time,temp,hum,region
0,속초,2019-01-01 01:00,-1.9,18.0,강원도
1,속초,2019-01-01 02:00,-2.2,17.0,강원도
2,속초,2019-01-01 03:00,-2.5,18.0,강원도
3,속초,2019-01-01 04:00,-3.9,20.0,강원도
4,속초,2019-01-01 05:00,-4.9,22.0,강원도


['강원도' '경기도' '인천광역시' '서울특별시' '경상북도' '충청북도' '충청남도' '대전광역시' '전라북도' '대구광역시'
 '울산광역시' '경상남도' '광주광역시' '부산광역시' '전라남도' '제주도' '세종특별자치시']


In [22]:
from datetime import datetime

In [23]:
population = pd.read_csv("data/population.csv", encoding="cp949", engine="python").T
population

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
행정구역(시군구)별,행정구역(시군구)별,전국,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,세종특별자치시,경기도,강원도,충청북도,충청남도,전라북도,전라남도,경상북도,경상남도,제주특별자치도
2019. 01,총인구수 (명),51826287,9766288,3438259,2460382,2955916,1459832,1488725,1154786,316814,13090648,1541693,1599155,2125797,1834532,1878904,2674005,3373214,667337
2019. 02,총인구수 (명),51829538,9770638,3436230,2458138,2956063,1459208,1487605,1153735,320326,13104696,1540445,1598868,2125372,1832227,1875862,2671587,3371016,667522
2019. 03,총인구수 (명),51833175,9770216,3434091,2455829,2956513,1458930,1486647,1152800,322693,13116579,1540488,1598775,2125778,1830499,1874232,2671036,3369949,668120
2019. 04,총인구수 (명),51836763,9766886,3431750,2454154,2956700,1458940,1485509,1152293,324417,13129508,1540794,1599488,2125912,1829273,1873183,2670375,3368933,668648
2019. 05,총인구수 (명),51840339,9762062,3429595,2452291,2957179,1459003,1484398,1151685,325777,13145482,1540776,1599854,2125732,1827871,1871718,2669731,3367857,669328
2019. 06,총인구수 (명),51842636,9757144,3427632,2450478,2957024,1459024,1482938,1151301,327643,13159178,1540855,1599650,2125855,1826717,1870587,2669850,3366989,669771
2019. 07,총인구수 (명),51845612,9751415,3425317,2448666,2956828,1459486,1481701,1150891,330298,13176011,1540714,1599368,2125349,1825381,1868856,2668836,3366286,670209
2019. 08,총인구수 (명),51847538,9744888,3423209,2446745,2956525,1459438,1480520,1150294,332098,13193943,1540780,1599402,2125160,1823982,1867094,2667657,3365282,670521
2019. 09,총인구수 (명),51849253,9740398,3421502,2445270,2956804,1459632,1479641,1149873,333757,13207219,1540721,1599339,2124855,1822883,1865915,2666795,3364039,670610


In [24]:
population.columns = population.iloc[0]
population = population.drop(population.index[0])

In [25]:
population

행정구역(시군구)별,행정구역(시군구)별,전국,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,세종특별자치시,경기도,강원도,충청북도,충청남도,전라북도,전라남도,경상북도,경상남도,제주특별자치도
2019. 01,총인구수 (명),51826287,9766288,3438259,2460382,2955916,1459832,1488725,1154786,316814,13090648,1541693,1599155,2125797,1834532,1878904,2674005,3373214,667337
2019. 02,총인구수 (명),51829538,9770638,3436230,2458138,2956063,1459208,1487605,1153735,320326,13104696,1540445,1598868,2125372,1832227,1875862,2671587,3371016,667522
2019. 03,총인구수 (명),51833175,9770216,3434091,2455829,2956513,1458930,1486647,1152800,322693,13116579,1540488,1598775,2125778,1830499,1874232,2671036,3369949,668120
2019. 04,총인구수 (명),51836763,9766886,3431750,2454154,2956700,1458940,1485509,1152293,324417,13129508,1540794,1599488,2125912,1829273,1873183,2670375,3368933,668648
2019. 05,총인구수 (명),51840339,9762062,3429595,2452291,2957179,1459003,1484398,1151685,325777,13145482,1540776,1599854,2125732,1827871,1871718,2669731,3367857,669328
2019. 06,총인구수 (명),51842636,9757144,3427632,2450478,2957024,1459024,1482938,1151301,327643,13159178,1540855,1599650,2125855,1826717,1870587,2669850,3366989,669771
2019. 07,총인구수 (명),51845612,9751415,3425317,2448666,2956828,1459486,1481701,1150891,330298,13176011,1540714,1599368,2125349,1825381,1868856,2668836,3366286,670209
2019. 08,총인구수 (명),51847538,9744888,3423209,2446745,2956525,1459438,1480520,1150294,332098,13193943,1540780,1599402,2125160,1823982,1867094,2667657,3365282,670521
2019. 09,총인구수 (명),51849253,9740398,3421502,2445270,2956804,1459632,1479641,1149873,333757,13207219,1540721,1599339,2124855,1822883,1865915,2666795,3364039,670610
2019. 10,총인구수 (명),51850705,9736289,3418871,2443528,2956984,1459360,1478436,1149409,336356,13218912,1540897,1599499,2124619,1821813,1865039,2666072,3363883,670738


In [26]:
pop = population[population.columns[1:]].reset_index(drop = True)
pop.columns.name = None
pop.index = pd.date_range('2019-01', '2020-08', freq='M').strftime('%Y-%m')
pop = pop.apply(pd.to_numeric)
pop

,전국,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,세종특별자치시,경기도,강원도,충청북도,충청남도,전라북도,전라남도,경상북도,경상남도,제주특별자치도
2019-01,51826287,9766288,3438259,2460382,2955916,1459832,1488725,1154786,316814,13090648,1541693,1599155,2125797,1834532,1878904,2674005,3373214,667337
2019-02,51829538,9770638,3436230,2458138,2956063,1459208,1487605,1153735,320326,13104696,1540445,1598868,2125372,1832227,1875862,2671587,3371016,667522
2019-03,51833175,9770216,3434091,2455829,2956513,1458930,1486647,1152800,322693,13116579,1540488,1598775,2125778,1830499,1874232,2671036,3369949,668120
2019-04,51836763,9766886,3431750,2454154,2956700,1458940,1485509,1152293,324417,13129508,1540794,1599488,2125912,1829273,1873183,2670375,3368933,668648
2019-05,51840339,9762062,3429595,2452291,2957179,1459003,1484398,1151685,325777,13145482,1540776,1599854,2125732,1827871,1871718,2669731,3367857,669328
2019-06,51842636,9757144,3427632,2450478,2957024,1459024,1482938,1151301,327643,13159178,1540855,1599650,2125855,1826717,1870587,2669850,3366989,669771
2019-07,51845612,9751415,3425317,2448666,2956828,1459486,1481701,1150891,330298,13176011,1540714,1599368,2125349,1825381,1868856,2668836,3366286,670209
2019-08,51847538,9744888,3423209,2446745,2956525,1459438,1480520,1150294,332098,13193943,1540780,1599402,2125160,1823982,1867094,2667657,3365282,670521
2019-09,51849253,9740398,3421502,2445270,2956804,1459632,1479641,1149873,333757,13207219,1540721,1599339,2124855,1822883,1865915,2666795,3364039,670610
2019-10,51850705,9736289,3418871,2443528,2956984,1459360,1478436,1149409,336356,13218912,1540897,1599499,2124619,1821813,1865039,2666072,3363883,670738


In [27]:
def devide(x):
    return x/pop['전국']
pop1 = pop.apply(lambda x: devide(x))

## 인구 집계
- 월별 전체 인구 집계 데이터 이용
    - 전국인구 수로 지역별 인구 수를 나눈 비율을 날씨 데이터 집계에 이용

In [28]:
pop1

,전국,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,세종특별자치시,경기도,강원도,충청북도,충청남도,전라북도,전라남도,경상북도,경상남도,제주특별자치도
2019-01,1.0,0.188443,0.066342,0.047474,0.057035,0.028168,0.028725,0.022282,0.006113,0.252587,0.029747,0.030856,0.041018,0.035398,0.036254,0.051596,0.065087,0.012876
2019-02,1.0,0.188515,0.066299,0.047427,0.057034,0.028154,0.028702,0.022260,0.006180,0.252842,0.029721,0.030849,0.041007,0.035351,0.036193,0.051546,0.065040,0.012879
2019-03,1.0,0.188493,0.066253,0.047379,0.057039,0.028147,0.028681,0.022241,0.006226,0.253054,0.029720,0.030845,0.041012,0.035315,0.036159,0.051531,0.065015,0.012890
2019-04,1.0,0.188416,0.066203,0.047344,0.057039,0.028145,0.028657,0.022229,0.006258,0.253286,0.029724,0.030856,0.041012,0.035289,0.036136,0.051515,0.064991,0.012899
2019-05,1.0,0.188310,0.066157,0.047305,0.057044,0.028144,0.028634,0.022216,0.006284,0.253576,0.029722,0.030861,0.041005,0.035260,0.036105,0.051499,0.064966,0.012911
2019-06,1.0,0.188207,0.066116,0.047268,0.057038,0.028143,0.028605,0.022208,0.006320,0.253829,0.029722,0.030856,0.041006,0.035236,0.036082,0.051499,0.064946,0.012919
2019-07,1.0,0.188086,0.066068,0.047230,0.057031,0.028151,0.028579,0.022198,0.006371,0.254139,0.029717,0.030849,0.040994,0.035208,0.036047,0.051477,0.064929,0.012927
2019-08,1.0,0.187953,0.066025,0.047191,0.057023,0.028149,0.028555,0.022186,0.006405,0.254476,0.029718,0.030848,0.040989,0.035180,0.036011,0.051452,0.064907,0.012933
2019-09,1.0,0.187860,0.065989,0.047161,0.057027,0.028151,0.028537,0.022177,0.006437,0.254723,0.029715,0.030846,0.040981,0.035157,0.035987,0.051434,0.064881,0.012934
2019-10,1.0,0.187775,0.065937,0.047126,0.057029,0.028145,0.028513,0.022168,0.006487,0.254942,0.029718,0.030848,0.040976,0.035136,0.035969,0.051418,0.064876,0.012936


In [29]:
pop1.to_csv("population_proportion.csv", index = False)

In [30]:
del pop1['전국']

In [ ]:
# result = pop1.copy()
# ['p'] = population['mul']/sum(population['mul'])
# display(pop1)
# print("sum of proportions : ", sum(pop1['p']))

In [59]:
# pop1 = pop.copy()
# # pop1 = pop.apply(get_proportion(pop))

In [31]:
pop1 = pop1.unstack().reset_index()
pop1.columns = ['region', 'month', 'p']
pop1.head()

,region,month,p
0,서울특별시,2019-01,0.188443
1,서울특별시,2019-02,0.188515
2,서울특별시,2019-03,0.188493
3,서울특별시,2019-04,0.188416
4,서울특별시,2019-05,0.188310


## Merge data
분당 데이터를 이용할 것이기 때문에 분단위로 나뉘어져 있는 형식은 우선 유지한다

In [32]:
mg = w1.groupby(['time', 'region']).mean().reset_index(drop = False)
mg['month'] = mg['time'].apply(lambda x: datetime.strptime(x,
                                                          "%Y-%m-%d %H:%M").strftime("%Y-%m"))

mg.head()

,time,region,temp,hum,month
0,2019-01-01 01:00,강원도,-7.692857,48.714286,2019-01
1,2019-01-01 01:00,경기도,-9.480000,66.000000,2019-01
2,2019-01-01 01:00,경상남도,-2.635714,44.000000,2019-01
3,2019-01-01 01:00,경상북도,-3.864286,42.642857,2019-01
4,2019-01-01 01:00,광주광역시,-1.500000,53.000000,2019-01


In [33]:
pd.merge(mg, pop1, on = ['month', 'region'])

,time,region,temp,hum,month,p
0,2019-01-01 01:00,강원도,-7.692857,48.714286,2019-01,0.029747
1,2019-01-01 02:00,강원도,-8.292857,50.142857,2019-01,0.029747
2,2019-01-01 03:00,강원도,-8.792857,53.357143,2019-01,0.029747
3,2019-01-01 04:00,강원도,-9.350000,53.857143,2019-01,0.029747
4,2019-01-01 05:00,강원도,-9.621429,56.285714,2019-01,0.029747
...,...,...,...,...,...,...
206049,2020-06-29 20:00,충청북도,20.720000,87.000000,2020-06,0.030818
206050,2020-06-29 21:00,충청북도,20.200000,88.400000,2020-06,0.030818
206051,2020-06-29 22:00,충청북도,19.880000,89.200000,2020-06,0.030818
206052,2020-06-29 23:00,충청북도,19.900000,89.200000,2020-06,0.030818


In [34]:
df = pd.merge(mg, pop1, on = ['month', 'region'])
df['TEMP'] = df['temp']*df['p']
df['HUM'] = df['hum']*df['p']
df1 = df[['time', 'region', 'TEMP', 'HUM']]
df1.head(10)

,time,region,TEMP,HUM
0,2019-01-01 01:00,강원도,-0.228842,1.449119
1,2019-01-01 02:00,강원도,-0.246690,1.491615
2,2019-01-01 03:00,강원도,-0.261564,1.587232
3,2019-01-01 04:00,강원도,-0.278137,1.602106
4,2019-01-01 05:00,강원도,-0.286212,1.674349
5,2019-01-01 06:00,강원도,-0.284937,1.731719
6,2019-01-01 07:00,강원도,-0.298111,1.810337
7,2019-01-01 08:00,강원도,-0.298961,1.867707
8,2019-01-01 09:00,강원도,-0.252427,1.810337
9,2019-01-01 10:00,강원도,-0.194845,1.627603


In [35]:
data = df1.groupby(['time']).sum().reset_index(drop=False)

In [36]:
data.head()

,time,TEMP,HUM
0,2019-01-01 01:00,-5.157666,53.621521
1,2019-01-01 02:00,-5.577352,56.876710
2,2019-01-01 03:00,-5.922292,58.845987
3,2019-01-01 04:00,-6.223195,60.195692
4,2019-01-01 05:00,-6.449101,61.929722


In [37]:
data.tail()

,time,TEMP,HUM
13099,2020-06-29 20:00,21.760203,86.624591
13100,2020-06-29 21:00,21.212699,89.290893
13101,2020-06-29 22:00,21.045186,89.997665
13102,2020-06-29 23:00,20.565999,92.057364
13103,2020-06-30 00:00,20.238832,93.233034


In [38]:
data.to_csv("월별인구가중합날씨_1.csv", encoding='utf-8', index = False)

In [39]:
pd.read_csv("월별인구가중합날씨_1.csv").head()

,time,TEMP,HUM
0,2019-01-01 01:00,-5.157666,53.621521
1,2019-01-01 02:00,-5.577352,56.876710
2,2019-01-01 03:00,-5.922292,58.845987
3,2019-01-01 04:00,-6.223195,60.195692
4,2019-01-01 05:00,-6.449101,61.929722
